In [1]:
import pandas as pd
import random
import os
import shutil

# --- ফাইল সেটআপ ---
filename = 'input_data.xlsx'

# ফাইল আছে কিনা চেক
if not os.path.exists(filename):
    print(f"Error: '{filename}' ফাইলটি পাওয়া যাচ্ছে না!")
    # এক্সিট করার জন্য
    raise SystemExit("Stop: File not found.")

print(f"File found: {filename}. Reading attempt started...")

df = None

# --- ধাপ ১: CSV হিসেবে পড়ার চেষ্টা (বিভিন্ন এনকোডিং দিয়ে) ---
encodings = ['utf-8', 'utf-8-sig', 'latin1', 'cp1252']
for enc in encodings:
    try:
        df = pd.read_csv(filename, encoding=enc)
        print(f"Success! CSV হিসেবে পড়া গেছে (Encoding: {enc})")
        break
    except:
        continue

# --- ধাপ ২: যদি CSV ফেল করে, তবে EXCEL হিসেবে পড়ার চেষ্টা ---
if df is None:
    print("Warning: CSV হিসেবে পড়া যাচ্ছে না। এটি সম্ভবত একটি Excel (.xlsx) ফাইল।")
    print("Excel ফরম্যাটে পড়ার চেষ্টা করা হচ্ছে...")
    
    try:
        # এক্সেল হিসেবে পড়ার জন্য ফাইলের নাম .xlsx হতে হয়, তাই আমরা কপি করে নাম বদলাব
        temp_excel_name = 'temp_input_file.xlsx'
        shutil.copy(filename, temp_excel_name)
        
        # এবার এক্সেল হিসেবে পড়ার চেষ্টা
        df = pd.read_excel(temp_excel_name)
        print("Success! ফাইলটি আসলে একটি Excel ফাইল ছিল। সফলভাবে পড়া হয়েছে।")
        
        # টেম্প ফাইল ডিলিট
        os.remove(temp_excel_name)
        
    except Exception as e:
        print(f"\n[CRITICAL ERROR] ফাইলটি কোনোভাবেই পড়া যাচ্ছে না। Error: {e}")
        print("পরামর্শ: মূল এক্সেল ফাইলটি খুলুন এবং 'Save As' -> 'CSV (Comma delimited)' ফরম্যাটে সেভ করে আবার আপলোড করুন।")
        raise SystemExit("Stop: Cannot read file.")

# --- ধাপ ৩: কলাম ঠিক করা এবং ডাটা প্রসেসিং ---
# কলাম নাম ক্লিন করা
df.columns = df.columns.str.strip().str.lower()

# কলাম ম্যাপ করা (English, Bangla, Marma)
# আপনার ফাইলে কলামের সম্ভাব্য নামগুলো খুঁজব
cols = df.columns.tolist()
print("Columns found:", cols)

english_col = next((c for c in cols if 'english' in c), None)
bangla_col = next((c for c in cols if 'bangla' in c or 'bengali' in c), None)
marma_col = next((c for c in cols if 'marma' in c), None)

if english_col and bangla_col and marma_col:
    df_clean = df[[english_col, bangla_col, marma_col]].dropna()
    df_clean.columns = ['English', 'Bangla', 'Marma']
else:
    # নাম না মিললে ইনডেক্স ধরে (0, 1, 3 - ধারণা করছি)
    print("Warning: কলামের নাম মিলছে না। ইনডেক্স অনুযায়ী কলাম নেওয়া হচ্ছে...")
    # সাধারণত ২য়, ৩য় বা ৪র্থ কলামে মারমা থাকে। আপনার ফাইল দেখে মনে হচ্ছে:
    # 0=English, 1=Bangla, 3=Marma (এটি ফাইলের গঠন অনুযায়ী বদলাতে পারে)
    try:
        df_clean = df.iloc[:, [0, 1, 3]].dropna() 
        df_clean.columns = ['English', 'Bangla', 'Marma']
    except:
         print("Error: কলাম খুঁজে পাওয়া যাচ্ছে না।")
         raise SystemExit("Stop: Column mismatch.")

# --- ধাপ ৪: ২০০০ বাক্য তৈরি (Augmentation) ---
def augment_text(text):
    text = str(text)
    words = text.split()
    if len(words) < 3: return text
    
    if random.random() > 0.5:
        # Swap
        try:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        except: return text
    else:
        # Delete
        if len(words) > 1:
            idx = random.randint(0, len(words)-1)
            words.pop(idx)
    return ' '.join(words)

generated_data = []
target_count = 2000
original_data = df_clean.to_dict('records')
generated_data.extend(original_data)

print(f"Original sentences: {len(original_data)}")
print("Generating new sentences...")

while len(generated_data) < target_count:
    row = random.choice(original_data)
    new_marma = augment_text(row['Marma'])
    generated_data.append({
        'English': row['English'],
        'Bangla': row['Bangla'],
        'Marma': new_marma
    })

# --- ধাপ ৫: ফাইল সেভ ---
output_filename = 'marma_dataset_2000.csv'
df_final = pd.DataFrame(generated_data)
# utf-8-sig দেওয়া হলো যাতে এক্সেলে বাংলা ঠিকমতো দেখা যায়
df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n[SUCCESS] কাজ সম্পন্ন! {len(df_final)} টি বাক্যের ফাইল তৈরি হয়েছে।")
print(f"ফাইলের নাম: {output_filename}")

File found: input_data.xlsx. Reading attempt started...
Excel ফরম্যাটে পড়ার চেষ্টা করা হচ্ছে...
Success! ফাইলটি আসলে একটি Excel ফাইল ছিল। সফলভাবে পড়া হয়েছে।
Columns found: ['english', 'standard bangla', 'garo', 'marma', 'chakma']
Original sentences: 292
Generating new sentences...

[SUCCESS] কাজ সম্পন্ন! 2000 টি বাক্যের ফাইল তৈরি হয়েছে।
ফাইলের নাম: marma_dataset_2000.csv


In [2]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# 1. NLTK এর প্রয়োজনীয় ফাইল ডাউনলোড (Colab এ এটি লাগবে)
nltk.download('vader_lexicon')

# 2. ফাইল লোড করা
filename = 'marma_dataset_2000.csv'
df = pd.read_csv(filename)

# 3. Sentiment Analyzer সেটআপ
sia = SentimentIntensityAnalyzer()

# ফাংশন: যা টেক্সট পড়ে লেবেল ঠিক করবে
def get_sentiment_label(text):
    if not isinstance(text, str):
        return 'ntr' # টেক্সট না থাকলে নিউট্রাল
    
    # স্কোর বের করা (-1 থেকে +1 এর মধ্যে)
    score = sia.polarity_scores(text)['compound']
    
    # লজিক সেট করা
    if score >= 0.05:
        return 'pos'  # Positive
    elif score <= -0.05:
        return 'neg'  # Negative
    else:
        return 'ntr'  # Neutral

# 4. English কলাম ব্যবহার করে লেবেল তৈরি করা
print("Labeling shuru hocche...")
# আপনার ফাইলে ইংরেজি কলামের নাম 'English' (বড় হাতের E) আছে কিনা দেখে নেবেন
# যদি ছোট হাতের হয় তবে row['english'] লিখবেন
df['label'] = df['English'].apply(get_sentiment_label)

# 5. উদাহরণ দেখা (Check)
print("\nSample Labeled Data:")
print(df[['English', 'label']].head(10))

# 6. ফাইল সেভ করা
output_file = 'marma_labeled_2000.csv'
df.to_csv(output_file, index=False)

print(f"\n[SUCCESS] Labeling complete! File saved as: {output_file}")
print("New Statistics:")
print(df['label'].value_counts())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\walton\AppData\Roaming\nltk_data...


Labeling shuru hocche...

Sample Labeled Data:
             English label
0        I hate you.   neg
1        I love you.   pos
2      I am in love.   pos
3        I am sorry.   neg
4   I am very sorry.   neg
5       How are you?   ntr
6         I am fine.   pos
7        I miss you.   neg
8  That is very bad.   neg
9     Get well soon.   pos

[SUCCESS] Labeling complete! File saved as: marma_labeled_2000.csv
New Statistics:
label
ntr    1155
pos     575
neg     270
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# ১. ফাইল লোড করা
filename = 'marma_labeled_2000.csv'
try:
    df = pd.read_csv(filename)
    df = df.dropna(subset=['Marma', 'label'])
except:
    print("Error: 'marma_labeled_2000.csv' ফাইলটি পাওয়া যাচ্ছে না!")
    exit()

# ২. ট্রেনিং ও টেস্ট ডাটা ভাগ করা
X = df['Marma']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ৩. মডেল পাইপলাইন
model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LinearSVC())
])

# ৪. ট্রেনিং
print("🤖 মডেল ট্রেনিং শুরু হচ্ছে...")
model_pipeline.fit(X_train, y_train)

accuracy = model_pipeline.score(X_test, y_test)
print(f"✅ ট্রেনিং সম্পন্ন! মডেল একুরেসি: {round(accuracy * 100, 2)}%")
print("-------------------------------------------------------")

# ৫. লাইভ টেস্টিং ফাংশন
def predict_marma_sentiment():
    print("\n👇 নিচে মারমা বাক্য লিখুন (বন্ধ করতে 'exit' লিখুন):")
    
    while True:
        # এখানে শুধু প্রম্পট থাকবে, বাক্য না
        user_text = input("\n📝 মারমা বাক্য: ") 
        
        if user_text.lower() == 'exit':
            print("ধন্যবাদ! প্রোগ্রাম বন্ধ করা হলো।")
            break
        
        if not user_text.strip():
            continue

        # প্রেডিকশন
        try:
            prediction = model_pipeline.predict([user_text])[0]
            
            result_map = {
                'pos': '🟢 POSITIVE (ভালো/ইতিবাচক)',
                'neg': '🔴 NEGATIVE (খারাপ/নেতিবাচক)',
                'ntr': '⚪ NEUTRAL (সাধারণ)'
            }
            print(f"রেজাল্ট: {result_map.get(prediction, prediction)}")
            
        except Exception as e:
            print("Error:", e)

# ফাংশন কল (বানান ঠিক করা হয়েছে)
predict_marma_sentiment()

🤖 মডেল ট্রেনিং শুরু হচ্ছে...
✅ ট্রেনিং সম্পন্ন! মডেল একুরেসি: 76.0%
-------------------------------------------------------

👇 নিচে মারমা বাক্য লিখুন (বন্ধ করতে 'exit' লিখুন):
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: 🟢 POSITIVE (ভালো/ইতিবাচক)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: 🔴 NEGATIVE (খারাপ/নেতিবাচক)


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# ১. ফাইল লোড
filename = 'marma_labeled_2000.csv'
df = pd.read_csv(filename)
df = df.dropna(subset=['Marma', 'label'])

# ২. ডাটা ভাগ করা
X = df['Marma']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) 
# test_size=0.15 মানে আমরা ট্রেনিংয়ে আরও বেশি ডাটা (৮৫%) দিচ্ছি।

# ৩. পাওয়ারফুল মডেল পাইপলাইন
model_pipeline = Pipeline([
    # ngram_range=(1, 3) মানে সে এখন ৩ শব্দের ফ্রেজও বুঝবে
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb', min_df=1)), 
    # C=10 এবং class_weight='balanced' একুরেসি বাড়াতে সাহায্য করে
    ('clf', LinearSVC(C=10, class_weight='balanced', max_iter=10000)) 
])

# ৪. ট্রেনিং
print("🚀 অ্যাডভান্সড ট্রেনিং শুরু হচ্ছে...")
model_pipeline.fit(X_train, y_train)

# ৫. রেজাল্ট চেক
predictions = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"🔥 নতুন একুরেসি: {round(accuracy * 100, 2)}%")
print("\nবিস্তারিত রিপোর্ট:")
print(classification_report(y_test, predictions))

🚀 অ্যাডভান্সড ট্রেনিং শুরু হচ্ছে...
🔥 নতুন একুরেসি: 99.0%

বিস্তারিত রিপোর্ট:
              precision    recall  f1-score   support

         neg       1.00      1.00      1.00        38
         ntr       0.98      1.00      0.99       170
         pos       1.00      0.97      0.98        92

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300



In [2]:
import pandas as pd
import random
from sklearn.utils import resample

# ১. ফাইল লোড করা
filename = 'marma_labeled_2000.csv'  # আপনার বর্তমান ফাইলের নাম

try:
    df = pd.read_csv(filename)
    # ফাকা ডাটা থাকলে ফেলে দিচ্ছি
    df = df.dropna(subset=['Marma', 'label'])
except:
    print("Error: ফাইলটি পাওয়া যাচ্ছে না! ফাইলের নাম চেক করুন।")
    exit()

print(f"Original Data Counts:\n{df['label'].value_counts()}")

# ২. অগমেন্টেশন ফাংশন (বাক্য মডিফাই করার জন্য)
def augment_text(text):
    text = str(text)
    words = text.split()
    if len(words) < 3: return text
    
    # ৫০% চান্স শব্দ অদলবদল করার, বাকি ৫০% চান্স শব্দ ডিলিট করার
    if random.random() > 0.5:
        try:
            # Swap (অদলবদল)
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        except: return text
    else:
        # Delete (মুছে ফেলা)
        if len(words) > 1:
            idx = random.randint(0, len(words)-1)
            words.pop(idx)
    return ' '.join(words)

# ৩. আলাদা আলাদা ক্লাসে ভাগ করা
df_pos = df[df['label'] == 'pos']
df_neg = df[df['label'] == 'neg']
df_ntr = df[df['label'] == 'ntr']

# ৪. ব্যালেন্স করার লজিক
def balance_class(df_class, target_count, label_name):
    current_count = len(df_class)
    
    if current_count >= target_count:
        # যদি টার্গেটের চেয়ে বেশি থাকে, তাহলে কমিয়ে ফেলব (Downsample)
        print(f"⬇️ Downsampling {label_name}: {current_count} -> {target_count}")
        return resample(df_class, n_samples=target_count, replace=False, random_state=42)
    
    else:
        # যদি টার্গেটের চেয়ে কম থাকে, তাহলে বাড়িয়ে ফেলব (Upsample / Augment)
        print(f"⬆️ Augmenting {label_name}: {current_count} -> {target_count}")
        
        # প্রথমে অরিজিনাল ডাটা রাখব
        augmented_data = df_class.to_dict('records')
        needed = target_count - current_count
        
        # যতগুলো দরকার ততগুলো নতুন বাক্য বানাব
        source_data = df_class.to_dict('records')
        while len(augmented_data) < target_count:
            row = random.choice(source_data)
            new_marma = augment_text(row['Marma'])
            
            # নতুন ডাটা লিস্টে যোগ করা
            augmented_data.append({
                'English': row['English'],
                'Bangla': row['Bangla'],
                'Marma': new_marma,
                'label': row['label'] # লেবেল একই থাকবে
            })
            
        return pd.DataFrame(augmented_data)

# ৫. টার্গেট সেট করা (মোট ২০০০)
# আমরা পজিটিভ ও নেগেটিভকে একটু বেশি প্রাধান্য দিচ্ছি যাতে মডেল ভালো শিখে
df_pos_balanced = balance_class(df_pos, 667, 'POS')
df_neg_balanced = balance_class(df_neg, 667, 'NEG')
df_ntr_balanced = balance_class(df_ntr, 666, 'NTR')

# ৬. সব জোড়া লাগানো
df_final = pd.concat([df_pos_balanced, df_neg_balanced, df_ntr_balanced])

# ডাটা শাফল করা (এলোমেলো করা)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# ৭. সেভ করা
output_filename = 'marma_balanced_2000.csv'
df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("\n------------------------------------------------")
print(f"[SUCCESS] নতুন ব্যালেন্সড ফাইল তৈরি হয়েছে: {output_filename}")
print("New Class Distribution:")
print(df_final['label'].value_counts())
print("Total:", len(df_final))

Original Data Counts:
label
ntr    1155
pos     575
neg     270
Name: count, dtype: int64
⬆️ Augmenting POS: 575 -> 667
⬆️ Augmenting NEG: 270 -> 667
⬇️ Downsampling NTR: 1155 -> 666

------------------------------------------------
[SUCCESS] নতুন ব্যালেন্সড ফাইল তৈরি হয়েছে: marma_balanced_2000.csv
New Class Distribution:
label
pos    667
neg    667
ntr    666
Name: count, dtype: int64
Total: 2000


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ১. ব্যালেন্স করা ফাইল লোড
filename = 'marma_balanced_2000.csv'

try:
    df = pd.read_csv(filename)
    df = df.dropna(subset=['Marma', 'label'])
    print(f"✅ ফাইল লোড হয়েছে: {filename}")
    print(f"মোট ডাটা: {len(df)}")
except:
    print(f"Error: '{filename}' ফাইলটি পাওয়া যাচ্ছে না। আগের কোডটি রান করে ফাইলটি তৈরি করুন।")
    exit()

# ২. ইনপুট ও আউটপুট ভাগ করা
X = df['Marma']
y = df['label']

# ডাটা স্প্লিট (Stratify ব্যবহার করছি যাতে টেস্ট ডাটায় সব ক্লাসের সমান অংশ থাকে)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# ৩. পাওয়ারফুল পাইপলাইন তৈরি
# ngram_range=(1, 3) -> ১ শব্দ, ২ শব্দ এবং ৩ শব্দের কম্বিনেশন দেখবে
# analyzer='char_wb' -> শব্দের ভেতরের অক্ষরের প্যাটার্নও দেখবে (মারমা ভাষার জন্য খুব কার্যকরী)
model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 4), analyzer='char_wb', min_df=1)),
    ('clf', LinearSVC(C=10, max_iter=10000, class_weight='balanced'))
])

# ৪. ট্রেনিং
print("\n🤖 মডেল ট্রেনিং শুরু হচ্ছে (Advanced Mode)...")
model_pipeline.fit(X_train, y_train)

# ৫. একুরেসি এবং রিপোর্ট
predictions = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("-------------------------------------------------------")
print(f"🏆 বর্তমান মডেল একুরেসি: {round(accuracy * 100, 2)}%")
print("-------------------------------------------------------")

print("\nডিটেইলড রিপোর্ট:")
print(classification_report(y_test, predictions))

print("\nConfusion Matrix (কোনটা আসলে কী, আর মডেল কী ভেবেছে):")
print(confusion_matrix(y_test, predictions, labels=['pos', 'neg', 'ntr']))

# ৬. লাইভ টেস্ট ফাংশন
def predict_now():
    print("\n👇 টেস্ট করার জন্য মারমা বাক্য লিখুন (বন্ধ করতে 'exit' লিখুন):")
    
    label_map = {
        'pos': '🟢 POSITIVE (ভালো)',
        'neg': '🔴 NEGATIVE (খারাপ)',
        'ntr': '⚪ NEUTRAL (সাধারণ)'
    }
    
    while True:
        user_input = input("\n📝 বাক্য: ")
        
        if user_input.lower() in ['exit', 'quit', 'stop']:
            print("ধন্যবাদ! বন্ধ করা হলো।")
            break
            
        if not user_input.strip():
            continue
            
        pred = model_pipeline.predict([user_input])[0]
        print(f"রেজাল্ট: {label_map.get(pred, pred)}")

# টেস্ট শুরু করুন
predict_now()

✅ ফাইল লোড হয়েছে: marma_balanced_2000.csv
মোট ডাটা: 2000

🤖 মডেল ট্রেনিং শুরু হচ্ছে (Advanced Mode)...
-------------------------------------------------------
🏆 বর্তমান মডেল একুরেসি: 98.67%
-------------------------------------------------------

ডিটেইলড রিপোর্ট:
              precision    recall  f1-score   support

         neg       1.00      1.00      1.00       100
         ntr       0.99      0.97      0.98       100
         pos       0.97      0.99      0.98       100

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300


Confusion Matrix (কোনটা আসলে কী, আর মডেল কী ভেবেছে):
[[ 99   0   1]
 [  0 100   0]
 [  3   0  97]]

👇 টেস্ট করার জন্য মারমা বাক্য লিখুন (বন্ধ করতে 'exit' লিখুন):
রেজাল্ট: 🔴 NEGATIVE (খারাপ)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: ⚪ NEUTRAL (সাধারণ)
রেজাল্ট: 🔴 NEGATIVE (খারাপ)
